In [2]:
from build import Genome, Config
from build.util.datetime import eta

import build as neat

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time as clock

In [3]:
DEVICE = 'cuda'
DTYPE = torch.float64

In [4]:
test_model = nn.Sequential(neat.NeatModule(3, 3, 10, 128, 3, 0, True, device=DEVICE, dtype=DTYPE, init_type='xavier_norm'))

TypeError: NeatModule.__init__() got multiple values for argument 'device'

In [ ]:
test_model

In [ ]:
test_modules = [m for m in test_model.modules() if isinstance(m, neat.NeatModule)]

In [ ]:
test_modules

In [ ]:
test_config = neat.Config('test')
test_config.species.compatibility_threshold = 0.5
test_config.general.fitness_criterion = 'max'
test_config.reproduction.elitism = 5
test_config.save()

In [ ]:
test_population = neat.Population(test_model, test_config, init_rep=True)

In [ ]:
test_population.pop_size

In [ ]:
test_input = torch.rand(200, 3).to(DEVICE, DTYPE)
test_output = test_input # torch.rand(100, 3).to(DEVICE, DTYPE)

In [33]:
a = torch.rand(100, 1, 7)
b = torch.rand(100, 7, 128)
x = torch.matmul(a, b)
c = torch.rand(100, 128, 128)
y = torch.matmul(x, c)
x.shape, y.shape

(torch.Size([100, 1, 128]), torch.Size([100, 1, 128]))

In [ ]:
a = torch.rand(20, 24, 1)
b = torch.mean(a, 1, True)
x = a - b
a.shape, b.shape, x.shape

In [ ]:
x = torch.rand(10, 10)
torch.max(x, 0)[1]

In [ ]:
torch.randn(10)

In [ ]:
1 - (torch.randint_like(torch.randn(10), 2) * 2)

In [32]:
test_mean = torch.zeros((10,))
test_std = torch.full((10,), 0.01)
torch.normal(test_mean, test_std)

tensor([ 0.0067,  0.0037,  0.0062, -0.0028, -0.0089, -0.0063, -0.0042, -0.0043,
         0.0077, -0.0069])

In [31]:
with torch.no_grad():
    test_pred = test_model(test_input[:2])
    print(test_pred.shape, test_input[:2].shape, ) #torch.mean(test_pred, dim=(0, 2)).shape, test_output.shape)

torch.Size([10, 2, 3]) torch.Size([2, 3])


In [32]:
with torch.no_grad():
    torch.cuda.empty_cache()
    test_pred = test_model(test_input).cpu()
    test_targ = test_output.unsqueeze(0).expand(10, -1, -1).cpu()
    print(torch.mean((test_targ - test_pred) ** 2, dim=(-1, -2)))

tensor([0.3734, 0.4537, 0.1525, 0.1401, 0.4220, 0.3056, 0.5086, 0.7334, 0.7414,
        0.5335], dtype=torch.float64)


In [33]:
test_opt = optim.Adam(test_model.parameters(), lr=1e-6, weight_decay=0)

In [34]:
def evaluate(genomes: dict[int, Genome], config: Config):
    test_model.cuda()
    # with torch.no_grad():
    mse = None
    ts = clock.perf_counter()
    units_total = 1000
    units_done = 0
    torch.cuda.empty_cache()
    for epoch in range(1000):
        units_done += 1

        pred = test_model(test_input)
        targ = test_output.unsqueeze(0).expand(*pred.shape)
        loss = F.mse_loss(targ, pred)

        test_opt.zero_grad()
        loss.backward()
        # nn.utils.clip_grad_norm_(test_model.parameters(), 1.0)
        test_opt.step()

        if epoch == 999:
            mse = torch.mean((targ - pred) ** 2, dim=(-1, -2))

        eta(ts, units_done, units_total, 'getting execution')
    print(f"\nmin = {torch.min(mse, dim=0)}             ")
    torch.cuda.empty_cache()
    test_model.cpu()

    mapping = {genome.index: genome for genome in genomes.values()}
    for index, genome in mapping.items():
        # genome.fitness = 1
        genome.fitness = -mse[index].item()

In [35]:
test_population.species.species[1].representative.gid

1

In [36]:
test_population.run(evaluate, 1)


 ****** Running generation 0 ****** 

Executing fitness function <function evaluate at 0x00000232C5676A20> on Population
getting execution: progress=100% eta=0sec
min = torch.return_types.min(
values=tensor(0.1365, device='cuda:0', dtype=torch.float64, grad_fn=<MinBackward0>),
indices=tensor(3, device='cuda:0'))             
Updating Population
Population's average fitness: -0.42929 std_dev: 0.19441
Best fitness: -0.13646 - size: (388, 33539) - species 1 - id 4
Getting Species
Finsished speciating in 0 sec% eta=0sec
Mean genetic distance 0.000, standard deviation 0.000
Population of 10 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
   1    0    10       --       --     0
Total extinctions: 0
Generation time: 4.073 sec


In [37]:
test_population.fitness_criterion

<function numpy.amax(a, axis=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)>

In [40]:
for I in range(10):
    evaluate(test_population.population, test_config)

getting execution: progress=100% eta=0sec
min = torch.return_types.min(
values=tensor(0.0096, device='cuda:0', dtype=torch.float64, grad_fn=<MinBackward0>),
indices=tensor(2, device='cuda:0'))             
getting execution: progress=100% eta=0sec
min = torch.return_types.min(
values=tensor(0.0095, device='cuda:0', dtype=torch.float64, grad_fn=<MinBackward0>),
indices=tensor(2, device='cuda:0'))             
getting execution: progress=100% eta=0sec
min = torch.return_types.min(
values=tensor(0.0095, device='cuda:0', dtype=torch.float64, grad_fn=<MinBackward0>),
indices=tensor(2, device='cuda:0'))             
getting execution: progress=100% eta=0sec
min = torch.return_types.min(
values=tensor(0.0094, device='cuda:0', dtype=torch.float64, grad_fn=<MinBackward0>),
indices=tensor(2, device='cuda:0'))             
getting execution: progress=100% eta=0sec
min = torch.return_types.min(
values=tensor(0.0094, device='cuda:0', dtype=torch.float64, grad_fn=<MinBackward0>),
indices=tensor(2, d

In [47]:
with torch.no_grad():
    test_model.cuda()
    test_pred = test_model(test_input[:2]).cpu()
    o = [test_input[:2], test_output[:2], test_pred[[0, 1, test_population.best_genome.index, test_population.population[3].index]] ]
    for p in o:
        print(p.cpu())

tensor([[0.9883, 0.4512, 0.2770],
        [0.4544, 0.2295, 0.5954]], dtype=torch.float64)
tensor([[0.9883, 0.4512, 0.2770],
        [0.4544, 0.2295, 0.5954]], dtype=torch.float64)
tensor([[[1.0096, 0.4558, 0.3095],
         [0.3296, 0.2027, 0.4050]],

        [[0.5454, 0.5022, 0.5358],
         [0.4682, 0.5172, 0.4699]],

        [[0.9845, 0.4393, 0.3095],
         [0.4764, 0.2990, 0.4051]],

        [[1.0092, 0.4559, 0.2853],
         [0.3314, 0.2019, 0.5469]]], dtype=torch.float64)
